# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

import pickle


In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table(table_name = 'InsertTableName', con=engine)

X = df.message
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity',
'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report', ]]

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
rf = RandomForestClassifier()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = rf)),#('clf', OneVsRestClassifier(MultiOutputClassifier(estimator = rf))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
y_pred.shape[1]

36

In [42]:
y_test.shape

(6554, 36)

In [11]:
# evaluate all steps on test set
y_pred = pipeline.predict(X_test)

for col in range(y_pred.shape[1]):
    print("Class: " + str(col))
    print(classification_report(y_test.iloc[:, col], y_pred[:, col]))

Class: 0
             precision    recall  f1-score   support

          0       0.61      0.34      0.44      1534
          1       0.82      0.93      0.87      4976
          2       0.75      0.07      0.12        44

avg / total       0.77      0.79      0.76      6554

Class: 1
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5445
          1       0.85      0.37      0.52      1109

avg / total       0.88      0.88      0.86      6554

Class: 2
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

Class: 3
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3820
          1       0.75      0.51      0.60      2734

avg / total       0.73      0.72      0.71      6554

Class: 4
             precision    recall  f1-score

/home/ian/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4] 
        }


cv =  GridSearchCV(pipeline, param_grid=parameters)
#cv.fit(X_train, y_train)

#best_clf = cv.best_estimator_



''' EXAMPLE FROM CASE STUDY
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    
    
'''

" EXAMPLE FROM CASE STUDY\n    parameters = {\n        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),\n        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),\n        'features__text_pipeline__vect__max_features': (None, 5000, 10000),\n        'features__text_pipeline__tfidf__use_idf': (True, False),\n        'clf__n_estimators': [50, 100, 200],\n        'clf__min_samples_split': [2, 3, 4],\n        'features__transformer_weights': (\n            {'text_pipeline': 1, 'starting_verb': 0.5},\n            {'text_pipeline': 0.5, 'starting_verb': 1},\n            {'text_pipeline': 0.8, 'starting_verb': 1},\n        )\n    }\n\n    cv = GridSearchCV(pipeline, param_grid=parameters)\n    \n    \n    \n"

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

Accuracy = TP+TN/TP+FP+FN+TN

Precision = TP/TP+FP

Recall = TP/TP+FN



In [21]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    precision = (y_pred == y_test).sum() / len(y_pred)
    recall = (y_pred == y_test).sum() / y_pred.sum()
    
    print("Labels:", labels)

    print("Accuracy:")
    print(accuracy)
    
    print("Precision:")
    print(precision)
    
    print("Recall:")
    print(recall)
    
    
    #print("\nBest Parameters:", cv.best_params)

    
display_results(cv, y_test, y_pred)

Labels: [0 1 2]
Accuracy:
related                   0.789136
request                   0.882362
offer                     0.995117
aid_related               0.722460
medical_help              0.919286
medical_products          0.953006
search_and_rescue         0.971315
security                  0.982453
military                  0.965670
child_alone               1.000000
water                     0.948581
food                      0.931797
shelter                   0.926152
clothing                  0.985505
money                     0.976350
missing_people            0.988251
refugees                  0.966738
death                     0.956515
other_aid                 0.868782
infrastructure_related    0.936832
transport                 0.954379
buildings                 0.952853
electricity               0.981080
tools                     0.994965
hospitals                 0.989167
shops                     0.995728
aid_centers               0.988862
other_infrastructure      0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
ada = AdaBoostClassifier()

pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = ada)),
])

# train classifier
pipeline2.fit(X_train, y_train)

# evaluate all steps on test set
y_pred2 = pipeline2.predict(X_test)



for col in range(y_pred2.shape[1]):
    print("Class: " + str(col))
    print(classification_report(y_test.iloc[:, col], y_pred2[:, col]))

print('OPTIMIZED ADABOOST MODEL')

parameters2 = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__learning_rate': [1.0, 1.2, 1.4, 1.6, 1.8] 
        }


cv2 =  GridSearchCV(pipeline2, param_grid=parameters2)
#best_clf2 = cv2.best_estimator_


display_results(cv2, y_test, y_pred2)

Class: 0
             precision    recall  f1-score   support

          0       0.58      0.24      0.34      1534
          1       0.80      0.94      0.86      4976
          2       0.27      0.14      0.18        44

avg / total       0.74      0.77      0.74      6554

Class: 1
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5445
          1       0.79      0.54      0.64      1109

avg / total       0.89      0.90      0.89      6554

Class: 2
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      0.99      0.99      6554

Class: 3
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3820
          1       0.76      0.61      0.67      2734

avg / total       0.76      0.75      0.75      6554

Class: 4
             precision    recall  f1-score

### 9. Export your model as a pickle file

In [35]:
pkl_filename = 'model.pkl'
pkl_outfile = open(pkl_filename,'wb')
pickle.dump(cv, pkl_outfile)
pkl_outfile.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.